In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import string

import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) - set(['at', 'do', 'your', 'from', 'to', 'out', 'no', 'the'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path = '/home/manikya_varshney/Documents/Python/Yale/All Graphs/final_processed_data/final_processed_april01.csv'
data = pd.read_csv(path, low_memory=False)

In [3]:
keywords = ['silver lining' , 'optimistic', 'hope', 
            'bright side', 'Safe', '#togetherapart']

##### 1. Casing (Upper or lower case)
##### 2. Noise Removal (Removal of punctuation, white spaces, special characters, HTML tags)
##### 3. Tokenization (Tweets to tokens i.e. words seprated by spaces)
##### 4. Stopword Removal
##### 5. Text Normalization (Stemming and Lemmatization)

In [4]:
#Convert to lower
for i in range(len(keywords)): 
    keywords[i] = keywords[i].lower()

#Remove punctuations   
for i in range(len(keywords)): 
    keywords[i] = keywords[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords)): 
    keywords[i] = keywords[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_tokens = [sub.split() for sub in keywords] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_filtered=remove_stopwords(keywords)

#Stemming
ps = PorterStemmer()
keywords_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_filtered]
keywords_stem = [" ".join(sentence) for sentence in keywords_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_filtered]
keywords_final = [" ".join(sentence) for sentence in keywords_lem]

## Fuzzy Matching

In [5]:
data['final'] = data['final'].apply(str)
choices = data['final'].tolist()

In [6]:
 def fuzzy_m(row):
        keyword_match, score = process.extractOne(row['final'], keywords_final, scorer = fuzz.partial_ratio)
        row['final_score'] = score
        row['final_keyword_match'] = keyword_match
        return row

In [7]:
interim_pe = data.apply(fuzzy_m, axis=1)

In [8]:
interim_pe

,user_id,text_duplicate,final,final_score,final_keyword_match
0,28433993,As Chicago households fill out 2020 census dur...,chicago household fill out 2020 census coronav...,50,safe
1,101043870,Most of the press corps is very busy covering ...,the press corp busy cover the really important...,64,bright side
2,16475267,@realDonaldTrump Mar 10–he was very specific: ...,mar 10–he specific google 1700 engineer work ...,55,bright side
3,16475267,"@realDonaldTrump , You were caustic &amp; sarc...",caustic amp sarcastic the rollout come even f...,50,hope
4,345397708,"Florida nears 8,000 coronavirus cases, as stat...",florida nears 8000 coronavirus case state repo...,54,togetherapart
...,...,...,...,...,...
16185,751644712570847236,If in 1938 Turkey’s government could produce a...,1938 turkey government could produce health m...,50,hope
16186,286628737,Sandy Medford was a friend of my family for de...,sandy medford friend family decade suspect hea...,55,bright side
16187,361010205,One month ago Trump claimed the number of Amer...,one month ago trump claimed the number america...,50,safe
16188,481389842,Yes. This. Especially if a substantial proport...,yes especially substantial proportion workforc...,50,hope


In [9]:
denominator = interim_pe.shape[0]

In [10]:
interim_pe['final_score'] = interim_pe['final_score'].astype(int)

In [11]:
interim_pe = interim_pe[interim_pe['final_score'] == 100]

In [12]:
interim_pe

,user_id,text_duplicate,final,final_score,final_keyword_match
14,823354136863252481,It is one thing to be optimistic or reassuring...,one thing to optimistic reassure entirely diff...,100,optimistic
27,29292924,Happy #NationalPoetryMonth - new pandemic poet...,happy nationalpoetrymonth new pandemic poetry ...,100,safe
33,92139131,I hope you get Coronavirus,hope get coronavirus,100,hope
49,298798289,Yeah! #Nantucket\nSTAY HOME SAFE LIVES\nCenter...,yeah nantucket stay home safe life center dise...,100,safe
53,911365226,@Zigmanfreud We often miss mild cases of the f...,often miss mild case the flu well hope the fin...,100,hope
...,...,...,...,...,...
16097,96454964,"Praying that our healthcare workers, on the fr...",pray healthcare worker the frontlines the figh...,100,safe
16153,805822016469110784,You really have to be unhinged to do this to y...,really to unhinged to do to your follower hope...,100,hope
16155,700473181308481536,@ASRA_Society @ESRA_Society \n\n#COVIDBlocks\n...,covidblocks choose the right procedure vigilan...,100,safe
16160,113667535,"MI has been home for me for last 5 years , i p...",mi home last 5 year pray amd declare the goodn...,100,safe


In [13]:
numerator = interim_pe.shape[0]

In [14]:
proportion = (numerator/denominator)
print(proportion)

0.03514515132798023
